<a href="https://colab.research.google.com/github/aso2001054/AI_teach2020/blob/master/2020AI0305_fishvideo_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import urllib.request as req
url = "https://github.com/masatokg/sample_photo/raw/master/fish.zip"
save_file = "fish.zip"
req.urlretrieve(url, save_file)

url = "https://github.com/masatokg/sample_photo/raw/master/nofish.zip"
save_file = "nofish.zip"
req.urlretrieve(url, save_file)

!unzip -o -q "./fish.zip"
!unzip -o -q "./nofish.zip"

In [2]:
import cv2
import os, glob, pickle
from sklearn.model_selection import train_test_split
from sklearn import datasets, metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [3]:
image_size = (64, 32)
path = "/content"
path_fish = path + "/fish"
path_nofish = "/nofish"
x = []
y = []
print(path_fish)
print(path_nofish)

/content/fish
/nofish


In [4]:
def read_dir(path, label, x, y, image_size):
  files = glob.glob(path + "/*.jpg")
  print(files)
  for f in files:
    img = cv2.imread(f)
    img = cv2.resize(img, image_size)
    img_data = img.reshape(-1,)
    x.append(img_data)
    y.append(label)

In [5]:
read_dir(path_nofish, 0, x, y, image_size)
read_dir(path_fish, 1, x, y, image_size)
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2)

clf = RandomForestClassifier()
clf.fit(x_train, y_train)

y_pred = clf.predict( x_test )
print(accuracy_score(y_test, y_pred))

with open( "fish.pkl", "wb") as fp:
  pickle.dump(clf, fp)

[]
['/content/fish/2827.jpg', '/content/fish/1326.jpg', '/content/fish/1352.jpg', '/content/fish/223.jpg', '/content/fish/1249.jpg', '/content/fish/1247.jpg', '/content/fish/2853.jpg', '/content/fish/1414.jpg', '/content/fish/2836.jpg', '/content/fish/1165.jpg', '/content/fish/1287.jpg', '/content/fish/3798.jpg', '/content/fish/3816.jpg', '/content/fish/1392.jpg', '/content/fish/274.jpg', '/content/fish/2626.jpg', '/content/fish/1283.jpg', '/content/fish/1944.jpg', '/content/fish/1308.jpg', '/content/fish/2814.jpg', '/content/fish/1231.jpg', '/content/fish/3197.jpg', '/content/fish/1421.jpg', '/content/fish/1988.jpg', '/content/fish/1922.jpg', '/content/fish/373.jpg', '/content/fish/2785.jpg', '/content/fish/1391.jpg', '/content/fish/1438.jpg', '/content/fish/1290.jpg', '/content/fish/3193.jpg', '/content/fish/1946.jpg', '/content/fish/2815.jpg', '/content/fish/4029.jpg', '/content/fish/1318.jpg', '/content/fish/276.jpg', '/content/fish/2634.jpg', '/content/fish/4030.jpg', '/content/fi

In [6]:
import cv2, os, copy, pickle
from google.colab.patches import cv2_imshow

with open( "fish.pkl", "rb") as fp:
  clf = pickle.load(fp)

  output_dir = "./bestshot"

  if not os.path.isdir(output_dir): os.mkdir(output_dir)
  

In [7]:
url = "https://github.com/masatokg/sample_photo/raw/master/fish.mp4"
save_file = "fish.mp4"
req.urlretrieve(url, save_file)

('fish.mp4', <http.client.HTTPMessage at 0x7f2ea674e908>)

In [8]:
cap = cv2.VideoCapture( save_file )

while(True):
  is_ok, frame = cap.read()
  if not is_ok:
    print("end")
    break

    frame = cv2.resize( frame, (640, 360) )

    frame2 = copy.copy( frame )
    frame_count += 1

    gray = cv2.cvtColor( frame, cv2.COLOR_BGR2GRAY )
    gray = cv2.GaussianBlur( gray, (15, 15), 0 )
    img_b = cv2.threshold( gray, 127, 255, cv2.THRESH_BINARY )[1]

    if not img_last is None:

      frame_diff = cv2.absdiff( img_last, img_b )
      cnts = cv2.findContours( frame_diff, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]

      fish_count = 0
      for pt in cnts:

        imgex = frame[ y:y+h, x:x+w ]
        imgex = cv2.resize( imgex, (64, 32) )
        image_data = imgex.reshape( -1, )

        pred_y = clf.predict([image_data])

        if pred_y[0] == 1:
          fish_count += 1
          cv2.rectangle( frame2, (x,y), (x+w, y+h), (0,255,0), 2 )

      if fish_count > fish_th:
        frame = output_dir + "/fish" + str(count) + ".jpg"
        cv2.imwrite( frame, frame )
        count += 1

        img_last = img_b

        cap.release()

        print( "ok", count, "/", frame_count, "/枚検出")


end
